In [4]:
import numpy as np
import pandas as pd
from pathlib import Path

In [5]:
box_folder = Path('/Users/dcutting/Library/CloudStorage/Box-Box/Exscientia - Bonvin Lab share')
benchmark_folder = box_folder/'benchmark_haddock_23_May_2023'
antigen_df = pd.read_csv('../data/AF2_antigen_rmsd.csv',index_col = 0)

In [6]:
paratope_df = pd.read_csv('../data/paratope_rmsds.csv',index_col=0)

In [10]:
paratope_df['rmsd_interface'] = np.NaN
paratope_df['rmsd_epitope'] = np.NaN
for pdbcode in paratope_df['pdb'].unique():
    paratope_res = set()
    epitope_res = set()
    with (benchmark_folder/f'{pdbcode}/{pdbcode}_constraint_pairs.txt').open('r') as file:
        for line in file.readlines()[1:]:
            antibody_line, antigen_line = line.split(':')
            paratope_res.add(''.join(antibody_line.split(',')[:2]))
            epitope_res.add(''.join(antigen_line.split(',')[:2]))
    n_paratope_res = len(paratope_res)
    n_epitope_res = len(epitope_res)
    n_interface_res = n_paratope_res+n_epitope_res
    epitope_rmsd = antigen_df.query('pdb==@pdbcode')['rmsd_epitope'].item()
    paratope_df.loc[paratope_df['pdb'] == pdbcode,'rmsd_interface'] = np.sqrt((paratope_df.loc[paratope_df['pdb']==pdbcode ,'rmsd_paratope']**2*n_paratope_res + epitope_rmsd**2*n_epitope_res)/n_interface_res)
    paratope_df.loc[paratope_df['pdb'] == pdbcode, 'rmsd_epitope'] = epitope_rmsd

In [11]:
paratope_df.to_csv('../data/para_epi_interface_rmsds.csv', index=False)